## Project Architecture

The whole architecture has evolved a lot over time. At first, we made a multithreading program where different tasks where running in parallel, eventhough python is not multithreading at the execution. This approach didn't offer significants performances and just made the project too complex. We opted for a simpler case where we simply have a main thread where the robot executes different tasks based on it's position and sensors.
Another problem we encountered was also the fact that after too many calls of a python element, the code crash. This is a protection for avoiding infinite loops. Our solution was simply calling `time.sleep()` before a new iteration.

Our class EventHandler in src/displacement/management.py is the core class where all the event are managed. It manages when to call at the right time Kalman, the global handler or the local handler.
In the constructor we initialize the class variables, the map, the camera, the kalman filter, the position and path for the robot.
The robot starts with the global handler. First of all, it checks if Kalman needs to be called, every `interval_camera` for the odometry with measurement and every `interval_odometry` for only the odometry. We recommend using a common factor between the two intervals.
After checking the next point of the path is used to computes the orientation and displacement needed , `delta_theta` and `delta_r`. Those values have constraints on them so that the robot does only relevant movements, it will rotate only if the rotation is bigger than `epsilon_theta` and will go forward if bigger than `epsilon_r`.
When displacing forward the robot checks if there are 3D obstacles in front of it. If one of the front sensors is bigger than the `obstacle_threshold` then the robot changes to the local avoidance mode. There it will starts avoiding the obstacle until it reaches the original path again. Then it will return the updated path and go back to global avoidance.
If the robot reaches in global avoidance a point of the path, it stops and delete the point of the path.
As long as there are still points to reach in the path the function `__global_handler` will be called again. If not the code will stop running because it will be the end of the program.

This simple architecture is very effective and simple for our task. We had first in mind a more complex idea where we used multithreading for every task. Unfortunately multithreading in python has a lot of downsides, like not beeing able to delete manually a thread and having no added performance using multithreading.
We tried to pause the threads and make them run again without success. Once the thread's flag is set on true we could not stop the thread until it reached the end.
The python interpreter is focus around making a safe code for the user and thus one of its characteristics is to run on a single thread, hence losing the purpose of multithreading performance.
This architecture was working well but too complex to manage, so we finally decided to the first explained architecture.

## Motion

The motion is manages at multiple places in the prgram. In the class `EventHandler`, in `self.__global_handler()` the robot has some speeds set for rotating and going forward. When avoiding obstacles in the class `ObstacleAvoidance` different speeds are also used.
All those speeds are based on the position of the robot on the map or relatively to the 3D obstacle. Those positions are constantly updated with kalman odometry with/without the measurements.

Now let's have a look at the Kalman in the file `src/kalman/kalman_filter.py`. The class called in the program is called `Kalmanhandler`, where the speeds, positions and useful variables are managed. The constructor initializes the `Kalman` class, the `SensorHandler` class and usefull class variables. The class offers the possibility to record on a parallel thread the speeds and compute the average when the function `get_kalman()` is called. The problem with this approach is that the class Thymio is called too often and the maximum depth is reached before the robot is able to reach the goal. If we redifine the thymio class in the code then using those functions pose no problem at all, except waiting a few seconds for the thymio to connect well.
So when `get_kalman()` is called, with or without speed recording, the program find an average speed or simply select the actual speed. That speed helps to compute the displacement done from the previous kalman call. The time interval from the previous kalman call is also computed and used.
Then a measurement is done if the measurement is needed.
Before calling the function `kalman_filter` from the class `Kalman`, we convert everything to meters and radians instead of centimeters and degrees and is converted back again after getting the estimated position.
`kalman_filter` takes the previous position, camera values and covariance to compute the newly estimated position and covariance. The first covariance is very small and will after a few calls converge to the real value.
After calling all the above functions, the program keeps on running and recomputes the angle and distance with its new estimated position.